Importing packages

In [1]:
import os
import json
from coupled_modeling import KnowledgeBase

Reading JSON data of an existing coupled system

In [3]:
input_path = os.path.join(
    os.path.dirname(os.path.realpath(__file__)),
    'mixed_fid_models/',
    'ProjectParametersCoSimFSI.json')

with open(input_path) as f:
    data = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'mixed_fid_models/ProjectParametersCoSimFSI.json'

Connecting to the knowledge base

In [ ]:
kb = KnowledgeBase('http://127.0.0.1:5000/api/v1.0/')

Importing data of the existing coupled system

In [ ]:
kb.import_coupled_kratos('mixed_fid_models', data)

Creating a new coupled system

In [ ]:
coupled_system = kb.create_coupled('Onera_FSI')

Copying problem data from the existing coupled system

In [ ]:
problem_data = kb.copy_instance(
    'instance_2',
    coupled_system,
    {'echo_level': 2})

Copying solver settings from the existing coupled system

In [ ]:
solver_settings = kb.copy_instance(
    'instance_3',
    coupled_system,
    {'echo_level': 4})

Creating convergence accelerators for the new coupled system from scratch

In [ ]:
convergence_accelerators = kb.create_instance(
    'convergence_accelerators',
    solver_settings,
    {'data_name': 'displacements', 'solver': 'CFD', 'type': 'aitken'})

Copying convergence criteria

In [ ]:
convergence_criteria = kb.copy_instance('instance_7', solver_settings, {
    'data_name': 'displacements',
    'solver': 'CFD',
    'type': 'relative_norm_initial_residual'})

Creating new coupling sequence

In [ ]:
coupling_sequence = kb.create_instance('coupling_sequence', solver_settings, {
    'name': 'CFD'})

Creating another coupling sequence

In [ ]:
coupling_sequence = kb.create_instance('coupling_sequence', solver_settings, {
    'name': 'SM'})

Copying input data list

In [ ]:
input_data_list = kb.copy_instance('instance_10', coupling_sequence, {
    'data_transfer_operator': 'mapping_operation',
    'from_solver': 'CFD',
    'data_transfer_operator_options': 'use_transpose'})

Creating output data list

In [ ]:
output_data_list = kb.create_instance('output_data_list', coupling_sequence, {
    'data': 'displacements',
    'data_transfer_operator': 'mapping_operation',
    'to_solver': 'CFD',
    'to_solver_data': 'displacements'})

Creating mapping operation

In [ ]:
mapping_operation = kb.create_instance('data_transfer_operators', solver_settings, {
    'label': 'mapping_operation',
    'type': 'kratos_mapping'})

Creating mapper settings

In [ ]:
mapper_settings = kb.create_instance('mapper_settings', mapping_operation, {
    'mapper_type': 'nearest_neighbor',
    'use_initial_configuration': True})

Copying solver

In [ ]:
CFD = kb.copy_instance('instance_12', solver_settings, {
    'label': 'CFD'})

Creating data

In [ ]:
displacements = kb.create_instance('data', CFD, {
    'label': 'displacements',
    'dimensions': 3,
    'model_part_name': 'WING',
    'variable_name': 'MESH_DISPLACEMENT'})

Creating another data

In [ ]:
lift_force = kb.create_instance('data', CFD, {
    'label': 'lift_force',
    'dimensions': 3,
    'model_part_name': 'WING',
    'variable_name': 'REACTION'})

Copying io settings

In [ ]:
io_settings = kb.copy_instance('instance_14', CFD, {
    'connect_to': 'run_SU2'})

Copying solver wrapper settings

In [ ]:
solver_wrapper_settings = kb.copy_instance('instance_13', CFD, {
    'export_data': 'displacements',
    'import_meshes': 'WING'})

Creating another solver

In [ ]:
SM = kb.create_instance('solvers', solver_settings, {
    'label': 'SM',
    'type': 'solver_wrappers.kratos.structural_mechanics_wrapper'})

Coping data for the second solver from the first one

In [ ]:
displacements = kb.copy_instance(displacements, SM, {
    'model_part_name': 'Structure.interface',
    'variable_name': 'DISPLACEMENT'})

Copying another data from the first solver

In [ ]:
lift_force = kb.copy_instance(lift_force, SM, {
    'model_part_name': 'Structure.interface',
    'variable_name': 'POINT_LOAD'})

Creating solver wrapper settings for the second solver

In [ ]:
solver_wrapper_settings = kb.create_instance('solver_wrapper_settings', SM, {
    'input_file': 'ProjectParametersSM'})

Exporting created coupled system in JSON

In [ ]:
export = kb.export_coupled_kratos(coupled_system)

with open('export_onera_fsi.json', 'w') as file:
    json.dump(export, file, indent=2)

Inferring new classes from the created coupled system

In [ ]:
kb.infer_coupled_structure(coupled_system)

Updating the knowledge base

In [ ]:
kb.save()